# How to run this?

`Handler` is a component to implement the logic. The easiest way to run it is with an `Executor`

In [4]:
from happyly.handling import Handler
from happyly.handling.types import ParsedMessage
from happyly.listening.executor import Executor


class MyHandler(Handler):

    def handle(self, _):
        print('Hello, world')

    def on_handling_failed(self, _, error: Exception):
        # notice that you have to explicitely handle errors, otherwise class will not be instanciated
        print(repr(error))


if __name__ == '__main__':
    executor = Executor(
        handler=MyHandler(),
    )

    executor.run()

Hello, world


# Executor? Handler? Why can't I just call my function!
For example, let's see how easy it is to use a subscriber for Google Pub/Sub mapped to your marshmallow message schema:

In [5]:
import marshmallow
from marshmallow import fields

from happyly.google_pubsub.deserializers import JSONDeserializerWithRequestIdRequired
from happyly.google_pubsub.subscribers import GooglePubSubSubscriber
from happyly.handling import Handler
from happyly.listening import Listener


class MySchema(marshmallow.Schema):
    request_id = fields.Str(required=True)
    info = fields.Str(required=True)


class MyHandler(Handler):

    def handle(self, message):
        print(f'Received message with {len(message)} attributes')
        print(f'request id is {message["request_id"]}')
        print(f'info is {message["info"]}')

    def on_handling_failed(self, message, error: Exception):
        print(repr(error))


if __name__ == '__main__':
    # set up auth if needed here

    listener = Listener(
        handler=MyHandler(),
        deserializer=JSONDeserializerWithRequestIdRequired(
            schema=MySchema(strict=True)
        ),
        subscriber=GooglePubSubSubscriber(
            project='machinaseptember2016',
            subscription_name='happyly_testing_01'
        )
    )
    future = listener.start_listening()
    try:
        future.result()
    except KeyboardInterrupt:
        future.cancel()


Received message with 2 attributes
request id is spam
info is eggs


# What is a `Listener`?
A `Listener` is an `Executor` with a `Subscriber`. Notice how you don't need to run `run` but just `start_listening`.

You can use a pre-made subscriber from happyly for certain Pub/Sub technology. Here we've used `GooglePubSubSubscriber`, but you can create your own by subclassing `Subscriber`.

# Why yet another component: a deserializer?
Imagine you switch from Google Pub/Sub to another tool. Or probably you change the way you store message attributes from JSON to protobuf. Not a problem - just take or implement another `Deserializer` and the `Handler` with actual logic stays unchanged! Notice that here:

In [ ]:
def handle(self, message):

message is just a Python dict. 

# Cool, we've got the Sub. What about the Pub?
Use the same things as you did before. Make `Handler.handle` return a dict, then add Serializer and Publisher to the `Listener`. Or to `Executor`. Let's see:

In [11]:
from collections import namedtuple

import marshmallow
from marshmallow import fields

from happyly.google_pubsub.deserializers import JSONDeserializerWithRequestIdRequired
from happyly.google_pubsub.publishers import GooglePubSubPublisher
from happyly.google_pubsub.serializers import BinaryJSONSerializer
from happyly.handling import Handler
from happyly.listening.executor import Executor


class InputSchema(marshmallow.Schema):
    request_id = fields.Str(required=True)


class OutputSchema(marshmallow.Schema):
    request_id = fields.Str(required=True)
    code = fields.Int(required=True)


class MyHandler(Handler):

    def handle(self, message):
        print('Publishing......')
        return {
            'request_id': message['request_id'],
            'code': 123,
        }

    def on_handling_failed(self, message, error: Exception):
        print(repr(error))


if __name__ == '__main__':
    # set up auth if needed here

    deserializer = JSONDeserializerWithRequestIdRequired(InputSchema(strict=True))
    serializer = BinaryJSONSerializer(OutputSchema(strict=True))

    listener = Executor(
        handler=MyHandler(),
        deserializer=deserializer,
        publisher=GooglePubSubPublisher(
            serializer=serializer,
            publish_all_to='happyly_testing',
            project='machinaseptember2016',
        )
    )
    msg = namedtuple('MessageMock', 'data')(b'{"request_id": "123"}')
    listener.run(msg)

Publishing......
